In [19]:
#https://github.com/srdeotarse/Linear-Regression-for-Stock-Chart-Pattern-Recognition/blob/master/TrianglePattern.ipynb

In [20]:
!pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [21]:
import pandas as pd
from pandas_datareader import data as web
import yfinance as yf
yf.pdr_override()
import numpy as np

STOCK = "RELIANCE.NS"
START_DATE = "2018-07-01"
END_DATE = "2022-07-07"
STOCK_PATTERN = "TRIANGLE"

df = web.get_data_yahoo(STOCK, start=START_DATE, end=END_DATE)
df['Id'] = np.arange(1, len(df)+1)
df.head(10)

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume,Id
Date,,,,,,,
2018-07-02 00:00:00+05:30,963.219666,963.219666,948.013794,951.579956,936.955200,6724054,1
2018-07-03 00:00:00+05:30,955.938660,969.262390,951.332336,962.179504,947.391785,5115293,2
2018-07-04 00:00:00+05:30,962.674805,982.685120,961.882324,980.802979,965.729126,5048125,3
2018-07-05 00:00:00+05:30,987.638184,998.931152,949.598755,955.443359,940.759216,18896793,4
2018-07-06 00:00:00+05:30,955.690979,973.769653,955.443359,968.370789,953.487976,9248105,5
2018-07-09 00:00:00+05:30,977.831116,990.609985,974.264954,987.737244,972.556763,5219197,6
2018-07-10 00:00:00+05:30,993.334167,1018.842407,990.609985,1016.068665,1000.452759,10543516,7
2018-07-11 00:00:00+05:30,1015.375244,1031.125977,1008.292419,1029.045654,1013.230286,8747208,8
2018-07-12 00:00:00+05:30,1034.543579,1088.680420,1032.809937,1072.186768,1055.708374,24337829,9


In [22]:
NUM_BEFORE = 3
NUM_AFTER = 3

def pivotId(df, candle, num_before, num_after):
    if candle-num_before < 0 or candle+num_after >= len(df):
        return 0
    
    pivotIdLow=1
    pivotIdHigh=1
    for i in range(candle-num_before, candle+num_after):
        if(df.Low[candle]>df.Low[i]):
            pivotIdLow=0
        if(df.High[candle]<df.High[i]):
            pivotIdHigh=0
    if pivotIdLow and pivotIdHigh:
        return 3
    elif pivotIdLow:
        return 1
    elif pivotIdHigh:
        return 2
    else:
        return 0
    
df['Pivot'] = df.apply(lambda row: pivotId(df, int(row.Id)-1, NUM_BEFORE, NUM_AFTER), axis=1)
df.tail(10)

,Open,High,Low,Close,Adj Close,Volume,Id,Pivot
Date,,,,,,,,
2022-06-23 00:00:00+05:30,2515.050049,2525.000000,2445.000000,2464.949951,2457.551025,8974475,982,1
2022-06-24 00:00:00+05:30,2480.000000,2511.000000,2468.000000,2500.050049,2492.545654,6571866,983,0
2022-06-27 00:00:00+05:30,2544.949951,2544.949951,2481.550049,2492.649902,2485.167725,7141493,984,0
2022-06-28 00:00:00+05:30,2482.100098,2533.949951,2482.100098,2527.699951,2520.112549,6263867,985,0
2022-06-29 00:00:00+05:30,2507.199951,2597.000000,2501.000000,2580.149902,2572.405029,8863172,986,0
2022-06-30 00:00:00+05:30,2580.000000,2615.899902,2566.000000,2595.649902,2587.858643,10666627,987,2
2022-07-01 00:00:00+05:30,2574.899902,2592.000000,2365.000000,2408.699951,2401.469727,37841671,988,1
2022-07-04 00:00:00+05:30,2378.949951,2442.000000,2366.100098,2413.699951,2406.454834,19317335,989,0
2022-07-05 00:00:00+05:30,2423.000000,2464.899902,2417.100098,2433.250000,2425.946045,8144239,990,0


In [23]:
def pointPosition(x):
    if x['Pivot']==1:
        return x['Low']-(0.01*df.High.max())
    elif x['Pivot']==2:
        return x['High']+(0.01*df.High.max())
    else:
        return np.nan

df['PointPosition'] = df.apply(lambda row: pointPosition(row), axis=1)
pd.set_option("display.max_rows", None, "display.max_columns", None)
display(df)

,Open,High,Low,Close,Adj Close,Volume,Id,Pivot,PointPosition
Date,,,,,,,,,
2018-07-02 00:00:00+05:30,963.219666,963.219666,948.013794,951.579956,936.955200,6724054,1,0,NaN
2018-07-03 00:00:00+05:30,955.938660,969.262390,951.332336,962.179504,947.391785,5115293,2,0,NaN
2018-07-04 00:00:00+05:30,962.674805,982.685120,961.882324,980.802979,965.729126,5048125,3,0,NaN
2018-07-05 00:00:00+05:30,987.638184,998.931152,949.598755,955.443359,940.759216,18896793,4,2,1027.492651
2018-07-06 00:00:00+05:30,955.690979,973.769653,955.443359,968.370789,953.487976,9248105,5,0,NaN
2018-07-09 00:00:00+05:30,977.831116,990.609985,974.264954,987.737244,972.556763,5219197,6,0,NaN
2018-07-10 00:00:00+05:30,993.334167,1018.842407,990.609985,1016.068665,1000.452759,10543516,7,0,NaN
2018-07-11 00:00:00+05:30,1015.375244,1031.125977,1008.292419,1029.045654,1013.230286,8747208,8,0,NaN
2018-07-12 00:00:00+05:30,1034.543579,1088.680420,1032.809937,1072.186768,1055.708374,24337829,9,0,NaN


In [24]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime

fig = go.Figure(data=[go.Candlestick(x=df.index,
                open=df['Open'],
                high=df['High'],
                low=df['Low'],
                close=df['Close'])])

fig.add_scatter(x=df.index, y=df['PointPosition'], mode="markers",
                marker=dict(size=5, color="MediumPurple"),
                name="Pivot")
#fig.update_layout(xaxis_rangeslider_visible=False)
fig.show()

In [25]:
from scipy.stats import linregress

RECENT_HIGH_PIVOT_POINT = 0
RECENT_LOW_PIVOT_POINT = 0

FEASIBLE_HIGH_PIVOT_POINTS = np.array([])
FEASIBLE_LOW_PIVOT_POINTS = np.array([])
FEASIBLE_HIGH = np.array([])
FEASIBLE_LOW = np.array([])

for i in reversed(range(len(df))):
    if df.Pivot[i] == 2 and RECENT_HIGH_PIVOT_POINT == 0:
        RECENT_HIGH_PIVOT_POINT = df.Id[i]
        FEASIBLE_HIGH = np.append(FEASIBLE_HIGH, df.High[i])
    if df.Pivot[i] == 1 and RECENT_LOW_PIVOT_POINT == 0:
        RECENT_LOW_PIVOT_POINT = df.Id[i]
        FEASIBLE_LOW = np.append(FEASIBLE_LOW, df.Low[i])
    if RECENT_HIGH_PIVOT_POINT != 0 and RECENT_LOW_PIVOT_POINT != 0:
        break

# Triangle Pattern
if STOCK_PATTERN == "TRIANGLE":
    try:
        FEASIBLE_HIGH_PIVOT_POINTS = np.append(FEASIBLE_HIGH_PIVOT_POINTS, RECENT_HIGH_PIVOT_POINT)
        FEASIBLE_LOW_PIVOT_POINTS = np.append(FEASIBLE_LOW_PIVOT_POINTS, RECENT_LOW_PIVOT_POINT)

        dfHigh=df[df['Pivot']==2]
        dfLow=df[df['Pivot']==1]

        MAX_HIGH_PIVOT_POINT = dfHigh.loc[dfHigh['PointPosition'] == dfHigh.PointPosition.max(), 'Id'].iloc[0]
        MIN_LOW_PIVOT_POINT = dfLow.loc[dfLow['PointPosition'] == dfLow.PointPosition.min(), 'Id'].iloc[0]

        FEASIBLE_HIGH_PIVOT_POINTS = np.append(FEASIBLE_HIGH_PIVOT_POINTS, MAX_HIGH_PIVOT_POINT)
        FEASIBLE_HIGH = np.append(FEASIBLE_HIGH, dfHigh.loc[dfHigh['PointPosition'] == dfHigh.PointPosition.max(), 'High'])
        FEASIBLE_LOW_PIVOT_POINTS = np.append(FEASIBLE_LOW_PIVOT_POINTS, MIN_LOW_PIVOT_POINT)
        FEASIBLE_LOW = np.append(FEASIBLE_LOW, dfLow.loc[dfLow['PointPosition'] == dfLow.PointPosition.min(), 'Low'])

        dfHigh = dfHigh[dfHigh['Id'] > MAX_HIGH_PIVOT_POINT]
        dfLow = dfLow[dfLow['Id'] > MIN_LOW_PIVOT_POINT]

        FEASIBLE_HIGH_PIVOT_POINTS = np.append(FEASIBLE_HIGH_PIVOT_POINTS, dfHigh.loc[dfHigh['PointPosition'] == dfHigh.PointPosition.max(), 'Id'].iloc[0])
        FEASIBLE_HIGH = np.append(FEASIBLE_HIGH, dfHigh.loc[dfHigh['PointPosition'] == dfHigh.PointPosition.max(), 'High'])
        FEASIBLE_LOW_PIVOT_POINTS = np.append(FEASIBLE_LOW_PIVOT_POINTS, dfLow.loc[dfLow['PointPosition'] == dfLow.PointPosition.min(), 'Id'].iloc[0])
        FEASIBLE_LOW = np.append(FEASIBLE_LOW, dfLow.loc[dfLow['PointPosition'] == dfLow.PointPosition.min(), 'Low']) 

        slmin, intercmin, rmin, pmin, semin = linregress(FEASIBLE_LOW_PIVOT_POINTS, FEASIBLE_LOW)
        slmax, intercmax, rmax, pmax, semax = linregress(FEASIBLE_HIGH_PIVOT_POINTS, FEASIBLE_HIGH)

        # To show stock data after END DATE to visualize stock trend after pattern
        df1 = web.get_data_yahoo(STOCK, start=END_DATE)
        df1['Id'] = np.arange(df.Id.max(), len(df1)+df.Id.max())

        df = pd.concat([df, df1])
        
        fig = go.Figure(data=[go.Candlestick(x=df.Id,
                        open=df['Open'],
                        high=df['High'],
                        low=df['Low'],
                        close=df['Close'])])

        fig.add_scatter(x=df.Id, y=df['PointPosition'], mode="markers",
                        marker=dict(size=5, color="MediumPurple"),
                        name="Pivot")

        fig.add_trace(go.Scatter(x=FEASIBLE_LOW_PIVOT_POINTS, y=slmin*FEASIBLE_LOW_PIVOT_POINTS + intercmin, mode='lines', line=dict(color="Blue"), name='min slope'))
        fig.add_trace(go.Scatter(x=FEASIBLE_HIGH_PIVOT_POINTS, y=slmax*FEASIBLE_HIGH_PIVOT_POINTS + intercmax, mode='lines', line=dict(color="Blue"), name='max slope'))

        fig.update_layout(
            font_size=20,
        )
        fig.show()

    except:
        print("No pattern detected")
        exit()

[*********************100%***********************]  1 of 1 completed


In [26]:
from scipy import stats
A,B,C,D,E = stats.linregress(FEASIBLE_HIGH_PIVOT_POINTS, FEASIBLE_HIGH)


print(C*C)
print(D)
print(C)

0.8188505022057843
0.2798855443106358
-0.9049035872432953
